Since the dawn of MOA countless souls have wondered  and mused upon the te separation between experiments. As dusk creeps on this competition, in a last effort to improve the correlation between CV and leaderboad score, a light shines in the darkness.


Here I show how to obtain 2 possible groupings using different cluster techiniqes
    - kmeans
    - hierarchical clustering
(Every experiment should include at most 6 trials with different time-dose combinations.)
I then proceed to use the latter because of its nicer separation properties.
I encourage those who come after me to try to improve this results and share their thoughts.

BTW Upvote if you find this useful  :)


In [ ]:
import sys 
sys.path.append('../input/iterative-stratification/iterative-stratification-master')

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sn


from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



import warnings
warnings.filterwarnings('ignore')

In [ ]:

from sklearn.metrics.pairwise import cosine_similarity,paired_distances ,paired_cosine_distances, pairwise_distances , pairwise_distances_argmin,pairwise_distances_argmin_min

from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering ,DBSCAN, KMeans

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
           # print("dtype after: ",props[col].dtype)
           # print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist


In [ ]:

def train_short_form_loader(feature_file,target_file,extra_target_file=None):
    '''takes the original target and features and creates a train dataset 
    in col long format'''


    train_features = pd.read_csv(feature_file)

    train_targets = pd.read_csv(target_file)
    train_features,_= reduce_mem_usage(train_features)
    train_targets,_ = reduce_mem_usage(train_targets)


    if extra_target_file is not None:
        extra_targets = pd.read_csv(extra_target_file)
        extra_targets,_ = reduce_mem_usage(extra_targets)
        train_targets = pd.merge(train_targets,extra_targets,on ='sig_id')
        del extra_targets

    targets = train_targets.columns[1:]

    train_melt=train_targets.merge(train_features,how="left",on="sig_id")


    del train_features,train_targets


    train_melt.set_index("sig_id",inplace=True)

    #train_melt["variable"]= train_melt["variable"].astype('category')
    train_melt["cp_type"]= train_melt["cp_type"].astype('category')
    train_melt["cp_dose"]= train_melt["cp_dose"].astype('category')

    return train_melt , targets.to_list()



def test_short_form_loader(feature_file):
    '''takes the original target and features and creates a train dataset 
    in col long format'''


    train_features = pd.read_csv(feature_file)

    #train_targets = pd.read_csv(target_file)
    train_features,_= reduce_mem_usage(train_features)
    #train_targets,_ = reduce_mem_usage(train_targets)

    train_melt =  train_features.copy()
    del train_features


    train_melt.set_index("sig_id",inplace=True)

    #train_melt["variable"]= train_melt["variable"].astype('category')
    train_melt["cp_type"]= train_melt["cp_type"].astype('category')
    train_melt["cp_dose"]= train_melt["cp_dose"].astype('category')

    return train_melt 

In [ ]:
train,target_cols = train_short_form_loader('../input/lish-moa/train_features.csv','../input/lish-moa/train_targets_scored.csv','../input/lish-moa/train_targets_nonscored.csv')

train.cp_type.unique()

train =  train.loc[train.cp_type=='trt_cp']

train.head()


In [ ]:
train_clus = train.drop(['cp_time','cp_dose','cp_type'],axis=1).copy()

#here we eight target more, the idea is that  trials from the same experiment should have similar targets

TARGETS_WEIGHT = 1000

train_clus[target_cols] = train_clus[target_cols] *TARGETS_WEIGHT

In [ ]:
kmeans=KMeans(n_clusters=3000,max_iter=100)


kmeans.fit(train_clus)

In [ ]:
import pickle as pk

pk.dump(kmeans,open('first_kmean_try.pk','wb'))

results_train_clus = train_clus.copy()
results_train_clus['kmeans1-clusters'] = kmeans.predict(train_clus)

results_train_clus['kmeans1-clusters'].value_counts()

In [ ]:
good_clusters = results_train_clus['kmeans1-clusters'].value_counts().loc[results_train_clus['kmeans1-clusters'].value_counts() < 7].index.to_list()

train['kmeans1-clusters'] = results_train_clus['kmeans1-clusters']

good_cluster_duplicates= train.loc[train['kmeans1-clusters'].isin(good_clusters),['kmeans1-clusters','cp_time','cp_dose'] ].groupby('kmeans1-clusters').apply(lambda x: x.duplicated().sum())

plt.hist(good_cluster_duplicates)

In [ ]:
plt.hist(results_train_clus['kmeans1-clusters'].value_counts(),bins=20)

unluckily the number of sig_ids per clusters does not peak at 5 or 6 as expected.

# Hierarchical clustering

## distance matrix creation

In [ ]:
distance_matrix= pairwise_distances(train_clus,metric='l1')

distance_matrix.shape

distance_matrix.tofile('distance_matrix_l1_pure')

distance_matrix


In [ ]:

train_partitions ={}

for key,group in train.groupby(['cp_dose','cp_time']):
    
    train_partitions[key]=group.drop(['cp_dose','cp_time'],axis=1)



Here we create a same time-dose penalization to add to the whole distance matrix,

In [ ]:
SAME_TIME_DOSE_PENALIZATION = 2000
penalization_same_group= train.apply(lambda x: SAME_TIME_DOSE_PENALIZATION * train.index.isin(train_partitions[tuple(x[['cp_dose','cp_time']].values.tolist())].index),axis=1)





In [ ]:
penalization_same_group.values

In [ ]:
distance_matrix = distance_matrix.reshape((len(train),len(train)))

mat_penalization = np.concatenate(penalization_same_group.values).reshape((len(train),len(train))) - 2000 * np.eye(len(train),len(train))

total_matrix = distance_matrix + mat_penalization

total_matrix.tofile('distance_matrix_l1_WsamegroupPenalization2')


In [ ]:

results_train_clus = train_clus.copy()
results_train_clus['aggclus1-clusters'] = aggclus.fit_predict(total_matrix)


In [ ]:
results_train_clus = pd.read_csv('../input/moafoldsmiscelanea/aggclus3_results.csv')

In [ ]:

results_train_clus['aggclus1-clusters'].value_counts()

In [ ]:
plt.hist(results_train_clus['aggclus1-clusters'].value_counts(),bins=20)

As you can well see hierarchical clustering using the distance matrix with the same time-dose penalization  gives a better distribution, in the sense that the majority of clusters ( experiments) involve 5 or 6 sig-ids (trials)

Unluckily I can not run the fll code on a kernel and each distance matrix file weights 3 GB so please tell me in the comments if you know of a way to create the distance matrix sequentially, or exploit some trick. 

Happy Kaggling !

P.S. : check my kernel on how to use this for Group kfolding